In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/prediction/feature_store_integration/prediction_feature_store_integration.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      查看GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/prediction/feature_store_integration/prediction_feature_store_integration.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>

## 概述

本笔记本演示了如何在 Vertex AI 上部署和提供模型，并集成特征存储。该集成使特征值能够在“预测”请求期间直接从特征存储中提取，而无需在预测请求中明确指定特征值。

**这是一个实验性版本**，受[Pre-GA提供](https://cloud.google.com/terms/service-terms)条款的覆盖范围，适用于您的Google Cloud Platform[服务条款](https://cloud.google.com/terms)。

实验功能专注于验证原型，并无法保证发布。它们不适用于生产环境，也不受任何SLA、支持义务或过时政策的覆盖，并可能会受到不兼容性变更的影响。它们的可靠性标准也可能不符合GA产品的标准。

在实验版本期间，使用该产品进行模型部署和提供是免费的：您仍将为其他GCP产品的使用产生费用，例如特征存储、存储等。

**在运行任何规模测试之前，请给我们留言。**

**如果您有任何问题或遇到任何问题，请随时联系vertexai-prediction-preview-feedback@google.com。**

项目需要被允许访问此功能。如果您有兴趣，请发送电子邮件到vertexai-prediction-preview-feedback@google.com，并提供您的项目编号或项目ID。

本笔记本中使用的数据集和模型基于此[预测和特征存储在线提供笔记本](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/feature_store/mobile_gaming/mobile_gaming_feature_store.ipynb)和[这篇博客文章](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml)。

## 简介

在此之前，当使用Vertex AI Feature Store来存储和获取用于预测的特征时，用户必须先调用Feature Store来获取特征，然后将这些特征发送到Vertex AI Prediction Service中进行预测请求。用户经常在将预测实例发送到预测容器之前或之后执行特征转换。这导致用户端需要进行大量的编排工作。


为了让我们的用户更容易地同时使用Feature Store和Prediction服务，我们提供了Feature Store和Prediction服务之间的内置集成。用户现在可以在上传模型到模型注册表时上传一个配置。这个配置包含了模型需要的特征信息，这些特征存储在Vertex AI Feature Store中，以及如何构建部署模型的预测请求。在预测路径上，用户只需提供Feature Store中存储的实体的ID，或者任何他们想要覆盖的特征。Vertex AI将负责读取所有特征，构建和执行预测请求，并返回最终的预测响应。


**我为什么要关心这个集成？**
- 只需通过一次调用在线预测服务进行预测，而不是调用两个单独的服务（一个或多个调用Feature Store和一个调用Prediction服务）
- 用户需要编写和维护的代码更少，因为他们不需要将Feature Store的响应转换为部署模型接受的格式。
- 发送预测请求到已部署模型的客户端无需担心模型接受的特征。因此，构建模型的数据科学团队可以继续进行实验，而无需让每个客户端在添加/删除/修改特征时更改他们的代码。因此，这将导致组织中更快的迭代。
- 模型-特征血统可以轻松明确地跟踪。

### 目标

目标是：
- 建立一个包含移动游戏用户人口统计和行为数据的特征存储。
- 学习如何为 Vertex Prediction 编写特征获取配置。
- 使用预先训练的流失模型将特征存储集成模型部署到 Vertex Prediction。
- 发送一个 `predict` 请求到部署的模型，实时从特征存储获取特征值。
- 学习如何使用身份模型调试一个特征存储集成模型。

### 数据集

该数据集是一款名为“Flood It!”的实际移动游戏应用的公共样本导出数据（[安卓](https://play.google.com/store/apps/details?id=com.labpixies.flood&pli=1)，[iOS](https://apps.apple.com/us/app/flood-it/id476943146)）。

### 模型

这个笔记本中使用的模型是基于[这篇博文](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml)的。该模型预测用户流失的概率。[原始数据](https://console.cloud.google.com/bigquery?p=firebase-public-project&d=analytics_153293282&t=events_20181003&page=table&_ga=2.214026135.-1049105954.1678419790)包含以下信息类别：

- 身份 - 独特的玩家身份编号。
- 人口统计特征 - 关于玩家的信息，比如玩家所在地理区域。
- 行为特征 - 玩家触发特定游戏事件的次数，比如达到新的等级。
- 流失倾向 - 这是标签或目标特征，提供了该玩家将流失的估计概率，即停止成为活跃玩家。

上面引用的博文解释了如何使用BigQuery存储原始数据，为机器学习预处理数据，并训练相应的模型。因为这个笔记本专注于Vertex Prediction，我们将重用该模型的预训练版本，可以从 `gs://featurestore_integration/model` 下载。

该模型按类别的特征列表：
- 人口统计特征：
  - `country`
  - `operating_system`
  - `language`
  - `user_pseudo_id`
- 行为特征：
  - `cnt_user_engagement`
  - `cnt_level_start_quickplay`
  - `cnt_level_end_quickplay`
  - `cnt_level_complete_quickplay`
  - `cnt_level_reset_quickplay`
  - `cnt_post_score`
  - `cnt_spend_virtual_currency`
  - `cnt_ad_reward`
  - `cnt_challenge_a_friend`
  - `cnt_completed_5_levels`
  - `cnt_use_extra_steps`
  - `month`
  - `julianday`
  - `dayofweek`
- 标签：
  - `churned`

### 成本

本教程使用Google Cloud的计费组件：

- Google Cloud存储
- Vertex AI 特征存储

了解 [Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

此功能不会为 Vertex AI 预测计费。

### 设置您的本地开发环境

**如果您正在使用 Vertex AI Workbench 笔记本**，则您的环境已经满足运行此笔记本的所有要求。 您可以跳过此步骤。

否则，请确保您的环境符合此笔记本的要求。
您需要以下内容：

* Docker
* Git
* Google Cloud SDK（gcloud）
* Python 3
* 虚拟环境
* 在使用 Python 3 的虚拟环境中运行的 Jupyter 笔记本

Google Cloud 的[设置Python开发环境指南](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了一个简化的指导：

1. [安装并初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

1. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)

1. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

1. 要安装Jupyter，请在终端窗口中的命令行中运行`pip install jupyter`。

1. 要启动Jupyter，请在终端窗口中的命令行中运行`jupyter notebook`。

1. 在Jupyter Notebook Dashboard中打开此笔记本。

### 安装额外的软件包

在您的笔记本环境中安装尚未安装的额外软件包依赖项。

In [ ]:
! pip install google-cloud-aiplatform

重新启动内核

在安装额外的软件包之后，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### 设立项目

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
shell_output = ! gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = shell_output[0]
print("Project Number:", PROJECT_NUMBER)

In [ ]:
! gcloud config set project $PROJECT_ID

将您选择使用的区域设置。

In [ ]:
REGION = ""

if REGION == "" or REGION is None:
    REGION = "us-central1"

时间戳

这个时间戳将用于预防资源名称冲突。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

###模型工件存储桶设置

创建一个 bucket 并将预训练模型复制到其中。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "-mobile-gaming-model-" + TIMESTAMP
    BUCKET_URI = f"gs://{BUCKET_NAME}"

只有您的存储桶尚不存在时：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

In [ ]:
! gsutil ls -al $BUCKET_URI

将经过训练的模型复制到您的存储桶中。

In [ ]:
ARTIFACT_URI = BUCKET_URI + "/model"

In [ ]:
! gsutil cp -r gs://featurestore_integration/model/* $ARTIFACT_URI

## 特征存储设置

首先，您需要设置一个特征存储来托管数据。

导入顶点AI库

In [ ]:
# Vertex AI and its Feature Store
from google.cloud import aiplatform
from google.cloud.aiplatform import Featurestore

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

创建特征存储。

In [ ]:
FEATURESTORE_ID = "mobile_gaming_" + TIMESTAMP  # @param {type:"string"}

# Vertex AI Feature store
ONLINE_STORE_NODES_COUNT = 5
DEMOGRAPHIC_ENTITY_ID = "demographic"
BEHAVIOR_ENTITY_ID = "behavior"
FEATURE_TIME = "timestamp"
ENTITY_ID_FIELD = "user_pseudo_id"

我们将使用在这篇[博客文章](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml)中使用的数据集的导出样本数据。

In [ ]:
try:
    mobile_gaming_feature_store = Featurestore.create(
        featurestore_id=FEATURESTORE_ID,
        online_store_fixed_node_count=ONLINE_STORE_NODES_COUNT,
        sync=True,
    )
except RuntimeError as error:
    print(error)
else:
    FEATURESTORE_RESOURCE_NAME = mobile_gaming_feature_store.resource_name
    print(f"Feature store created: {FEATURESTORE_RESOURCE_NAME}")

### 创建实体

In [ ]:
try:
    demographic_entity_type = mobile_gaming_feature_store.create_entity_type(
        entity_type_id=DEMOGRAPHIC_ENTITY_ID,
        description="User demographic Entity",
        sync=True,
    )
except RuntimeError as error:
    print(error)
else:
    DEMOGRAPHIC_ENTITY_RESOURCE_NAME = demographic_entity_type.resource_name
    print("Entity type name is", DEMOGRAPHIC_ENTITY_RESOURCE_NAME)

In [ ]:
try:
    behavior_entity_type = mobile_gaming_feature_store.create_entity_type(
        entity_type_id=BEHAVIOR_ENTITY_ID, description="User behavior Entity", sync=True
    )
except RuntimeError as error:
    print(error)
else:
    BEHAVIOR_ENTITY_RESOURCE_NAME = behavior_entity_type.resource_name
    print("Entity type name is", BEHAVIOR_ENTITY_RESOURCE_NAME)

### 创建特征

特性配置

In [ ]:
demographic_feature_configs = {
    "country": {
        "value_type": "STRING",
        "description": "The country of customer",
        "labels": {"status": "passed"},
    },
    "operating_system": {
        "value_type": "STRING",
        "description": "The operating system of device",
        "labels": {"status": "passed"},
    },
    "language": {
        "value_type": "STRING",
        "description": "The language of device",
        "labels": {"status": "passed"},
    },
    "user_pseudo_id": {
        "value_type": "STRING",
        "description": "User pseudo id",
        "labels": {"status": "passed"},
    },
}

behavior_feature_configs = {
    "cnt_user_engagement": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement level",
        "labels": {"status": "passed"},
    },
    "cnt_level_start_quickplay": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement with start level",
        "labels": {"status": "passed"},
    },
    "cnt_level_end_quickplay": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement with end level",
        "labels": {"status": "passed"},
    },
    "cnt_level_complete_quickplay": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement with complete status",
        "labels": {"status": "passed"},
    },
    "cnt_level_reset_quickplay": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement with reset status",
        "labels": {"status": "passed"},
    },
    "cnt_post_score": {
        "value_type": "DOUBLE",
        "description": "A variable of user score",
        "labels": {"status": "passed"},
    },
    "cnt_spend_virtual_currency": {
        "value_type": "DOUBLE",
        "description": "A variable of user virtual amount",
        "labels": {"status": "passed"},
    },
    "cnt_ad_reward": {
        "value_type": "DOUBLE",
        "description": "A variable of user reward",
        "labels": {"status": "passed"},
    },
    "cnt_challenge_a_friend": {
        "value_type": "DOUBLE",
        "description": "A variable of user challenges with friends",
        "labels": {"status": "passed"},
    },
    "cnt_completed_5_levels": {
        "value_type": "DOUBLE",
        "description": "A variable of user level 5 completed",
        "labels": {"status": "passed"},
    },
    "cnt_use_extra_steps": {
        "value_type": "DOUBLE",
        "description": "A variable of user extra steps",
        "labels": {"status": "passed"},
    },
    "month": {
        "value_type": "INT64",
        "description": "First touch month",
        "labels": {"status": "passed"},
    },
    "julianday": {
        "value_type": "INT64",
        "description": "First touch julian day",
        "labels": {"status": "passed"},
    },
    "dayofweek": {
        "value_type": "INT64",
        "description": "First touch day of week",
        "labels": {"status": "passed"},
    },
}

使用`batch_create_features`方法创建特征。

In [ ]:
try:
    demographic_entity_type.batch_create_features(
        feature_configs=demographic_feature_configs, sync=True
    )
except RuntimeError as error:
    print(error)
else:
    for feature in demographic_entity_type.list_features():
        print("")
        print(f"The resource name of {feature.name} feature is", feature.resource_name)

In [ ]:
try:
    behavior_entity_type.batch_create_features(
        feature_configs=behavior_feature_configs, sync=True
    )
except RuntimeError as error:
    print(error)
else:
    for feature in behavior_entity_type.list_features():
        print("")
        print(f"The resource name of {feature.name} feature is", feature.resource_name)

摄入功能

首先将数据复制到您的存储桶中。

In [ ]:
SOURCE_URI = BUCKET_URI + "/feature_store_source/mobile_gaming_dataset.csv"

In [ ]:
! gsutil cp gs://featurestore_integration/data/mobile_gaming_dataset.csv $SOURCE_URI

In [ ]:
DEMOGRAPHIC_FEATURES_IDS = [
    feature.name for feature in demographic_entity_type.list_features()
]

In [ ]:
# This may take several minutes to run

try:
    demographic_entity_type.ingest_from_gcs(
        feature_ids=DEMOGRAPHIC_FEATURES_IDS,
        feature_time=FEATURE_TIME,
        gcs_source_uris=SOURCE_URI,
        gcs_source_type="csv",
        entity_id_field=ENTITY_ID_FIELD,
        disable_online_serving=False,
        worker_count=10,
        sync=True,
    )
except RuntimeError as error:
    print(error)

In [ ]:
BEHAVIOR_FEATURES_IDS = [
    feature.name for feature in behavior_entity_type.list_features()
]

In [ ]:
# This may take several minutes to run

try:
    behavior_entity_type.ingest_from_gcs(
        feature_ids=BEHAVIOR_FEATURES_IDS,
        feature_time=FEATURE_TIME,
        gcs_source_uris=SOURCE_URI,
        gcs_source_type="csv",
        entity_id_field=ENTITY_ID_FIELD,
        disable_online_serving=False,
        worker_count=10,
        sync=True,
    )
except RuntimeError as error:
    print(error)

现在我们已经建立了我们的特征存储，让我们尝试为一个样本实体获取特征。

In [ ]:
demographic_entity_type.read(entity_ids="AB0F2EE5F9F401763BE1E9FA55410312")

In [ ]:
behavior_entity_type.read(entity_ids="AB0F2EE5F9F401763BE1E9FA55410312")

## 为顶点预测提取配置文件

为了启用模型部署的特征获取功能，文件 `prediction_featurestore_fetch_config.yaml` 必须包含在 `artifacts_uri` 路径中。

这个 yaml 文件的目的是指定以下信息：
- 要从哪个**特征存储**（位置、项目 ID、特征存储 ID）中获取特征。
- 与要获取的特征对应的**特征名称**（即特征 ID）。
- 如何从用户提供的预测请求和获取的特征构建**预测请求**。
- 在预测时，从哪些**实体**中获取特征。

该 yaml 文件必须符合以下协议。

### 特性获取配置协议：

```protobuf
message FeatureFetchConfig {
  // 在获取特征后自动生成的内部预测请求的格式。目前预测支持XGBoost，TensorFlow，
  // scikit-learn，Pytorch和自定义容器。在这些框架中，XGBoost只支持数组输入格式（即输入特征以数组形式呈现），
  // 而其他三个框架可以同时支持字典格式和数组格式的输入。
  ModelInputFormat model_input_format = 3;
  enum ModelInputFormat {
    MODEL_INPUT_FORMAT_UNSPECIFIED = 0;
    ARRAY = 1;
    DICT = 2;
  }

  message FeatureSource {
    // 必填。实体类型的完全限定名称，例如：
    // "projects/acme/locations/us-central1/featurestores/fs/entityTypes/movies"
    string entity_type = 2;

    // 必填。指定用户发送的请求中（而不是包含特征值的自动生成的预测请求）
    // 包含要从此源提取的实体ID的字段的名称。用户发送的输入请求是一个JSON字典。
    // 要提取的实体ID由具有此键的字典条目持有。
    string entity_id_field = 3;
  }

  // 特性源定义。此映射的键用于在特性设置中引用该源。键不能为空。
  map<string, FeatureSource> feature_sources = 5;

  // 指定预测输入的细节
  repeated Feature features = 4;
  message Feature {
    // 废弃字段。
    reserved 1, 2;

    // 当 internal_request_format = DICT 时，该名称用作
    // 内部预测请求的关键特征值。在 FeatureFetchConfig 中，透传特性可以由
    // 仅带有名称的 Feature 消息代表。
    string name = 3;

    message FetchFrom {
      // 必填。从中提取特征值的特征源的名称。
      // 如果特征值来自原始请求（透传），则留空。
      string source_name = 1;

      // 必填。要提取的特征ID。这应该是与上述 featureSource 对应的实体类型的有效特征ID。
      // 如果使用 featureSource，必须提供。
      string feature_id = 2;
    }
    // 定义提取特征值的源。请注意，如果用户发送的请求已经为此特性提供了值，那个值将被
    // 直接使用（即该设置将被忽略，并且对于该请求，该特性将被视为透传）如果未设置此字段，
    // 则特性值始终从原始请求中透传。
    FetchFrom fetch_from = 4;
  }
}
```

### 写特征提取配置

请注意以下事项
- 我们使用`modelInputFormat: DICT`，因为模型需要字典输入
- `user_pseudo_id`是一个直通特征。用户预测请求中的键值对将直接传递给模型服务器。
- 所有其他具有非空`fetchFrom`的`features`将在预测时从特征存储中提取。

In [ ]:
FEATURE_FETCH_CONFIG_TEMPLATE = """modelInputFormat: DICT
featureSources:
  user:
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/demographic
    entityIdField: demographic
  behavior:
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    entityIdField: behavior
features:
- name: user_pseudo_id
- name: country
  fetchFrom:
    sourceName: user
    featureId: country
- name: operating_system
  fetchFrom:
    sourceName: user
    featureId: operating_system
- name: language
  fetchFrom:
    sourceName: user
    featureId: language
- name: cnt_user_engagement
  fetchFrom:
    sourceName: behavior
    featureId: cnt_user_engagement
- name: cnt_level_start_quickplay
  fetchFrom:
    sourceName: behavior
    featureId: cnt_level_start_quickplay
- name: cnt_level_end_quickplay
  fetchFrom:
    sourceName: behavior
    featureId: cnt_level_end_quickplay
- name: cnt_level_complete_quickplay
  fetchFrom:
    sourceName: behavior
    featureId: cnt_level_complete_quickplay
- name: cnt_level_reset_quickplay
  fetchFrom:
    sourceName: behavior
    featureId: cnt_level_reset_quickplay
- name: cnt_post_score
  fetchFrom:
    sourceName: behavior
    featureId: cnt_post_score
- name: cnt_spend_virtual_currency
  fetchFrom:
    sourceName: behavior
    featureId: cnt_spend_virtual_currency
- name: cnt_ad_reward
  fetchFrom:
    sourceName: behavior
    featureId: cnt_ad_reward
- name: cnt_challenge_a_friend
  fetchFrom:
    sourceName: behavior
    featureId: cnt_challenge_a_friend
- name: cnt_completed_5_levels
  fetchFrom:
    sourceName: behavior
    featureId: cnt_completed_5_levels
- name: cnt_use_extra_steps
  fetchFrom:
    sourceName: behavior
    featureId: cnt_use_extra_steps
- name: month
  fetchFrom:
    sourceName: behavior
    featureId: month
- name: julianday
  fetchFrom:
    sourceName: behavior
    featureId: julianday
- name: dayofweek
  fetchFrom:
    sourceName: behavior
    featureId: dayofweek"""

feature_fetch_config = FEATURE_FETCH_CONFIG_TEMPLATE.format(
    PROJECT_NUMBER=PROJECT_NUMBER, REGION=REGION, FEATURESTORE_ID=FEATURESTORE_ID
)

with open("prediction_featurestore_fetch_config.yaml", "w") as f:
    f.write(feature_fetch_config)

将特性获取配置yaml复制到工件的gcs路径。

In [ ]:
!gsutil cp prediction_featurestore_fetch_config.yaml $ARTIFACT_URI

## 上传和部署模型到Vertex预测

### 上传模型

In [ ]:
DISPLAY_NAME = "mobile_gaming_featureStore_integration_" + TIMESTAMP

model = aiplatform.Model.upload(
    display_name=DISPLAY_NAME,
    artifact_uri=ARTIFACT_URI,
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest",
    sync=False,
)

model.wait()

### 部署模型

#### 预测服务账户

由于预测工作负载的默认标识没有访问 Feature Store 的权限，我们需要为这个新功能提供一个服务账户。该服务账户需要在您的项目中拥有 `Vertex AI Feature Store 数据查看器` 权限。

在运行以下命令之前，请打开终端并运行 `gcloud auth login`。

In [ ]:
SA_NAME = "prediction-feature-store-fetch"
SA_DESCRIPTION = '"Fetch feature from Feature Store during Prediction"'
DISPLAY_NAME = "prediction-feature-store-fetch"

!gcloud iam service-accounts create $SA_NAME \
    --description=$SA_DESCRIPTION \
    --display-name=$DISPLAY_NAME

In [ ]:
SERVICE_ACCOUNT = f"{SA_NAME}@{PROJECT_ID}.iam.gserviceaccount.com"

授予Service Account Vertex AI Feature Store Data Viewer角色来访问特征存储中的数据，在预测期间。

In [ ]:
!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:$SERVICE_ACCOUNT \
    --role=roles/aiplatform.featurestoreDataViewer;

现在我们准备部署模型。

In [ ]:
DEPLOYED_NAME = DISPLAY_NAME + TIMESTAMP

TRAFFIC_SPLIT = {"0": 100}

MACHINE_TYPE = "n1-standard-4"

MIN_NODES = 1
MAX_NODES = 1

endpoint = model.deploy(
    deployed_model_display_name=DEPLOYED_NAME,
    traffic_split=TRAFFIC_SPLIT,
    machine_type=MACHINE_TYPE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    service_account=SERVICE_ACCOUNT,
)

### 预测

TF模型期望输入的格式如下：

In [ ]:
default_pred_request = [
    {
        "user_pseudo_id": "AB0F2EE5F9F401763BE1E9FA55410312",
        "country": "Australia",
        "operating_system": "IOS",
        "language": "en-au",
        "cnt_user_engagement": 3.0,
        "cnt_level_start_quickplay": 1.0,
        "cnt_level_end_quickplay": 0.0,
        "cnt_level_complete_quickplay": 0.0,
        "cnt_level_reset_quickplay": 0.0,
        "cnt_post_score": 0.0,
        "cnt_spend_virtual_currency": 0.0,
        "cnt_ad_reward": 0.0,
        "cnt_challenge_a_friend": 0.0,
        "cnt_completed_5_levels": 0.0,
        "cnt_use_extra_steps": 0.0,
        "month": 7,
        "julianday": 194,
        "dayofweek": 6,
    },
]

不直接在预测请求中提供特征值，而是指定实体ID键（例如`人口统计学`和`行为`）和实体ID值（例如`AB0F2EE5F9F401763BE1E9FA55410312`）。特征值将在预测时由服务端获取。

In [ ]:
fs_pred_request = [
    {
        "user_pseudo_id": "AB0F2EE5F9F401763BE1E9FA55410312",
        "demographic": "AB0F2EE5F9F401763BE1E9FA55410312",
        "behavior": "AB0F2EE5F9F401763BE1E9FA55410312",
    },
]

endpoint.predict(fs_pred_request)

特性商店抓取的任何特性子集都可以被覆盖。例如，

In [ ]:
fs_pred_request_with_overridden_features = [
    {
        "user_pseudo_id": "AB0F2EE5F9F401763BE1E9FA55410312",
        "demographic": "AB0F2EE5F9F401763BE1E9FA55410312",
        "behavior": "AB0F2EE5F9F401763BE1E9FA55410312",
        "cnt_ad_reward": 10.0,
        "cnt_challenge_a_friend": 10.0,
        "cnt_completed_5_levels": 10.0,
        "cnt_use_extra_steps": 10.0,
    },
]

endpoint.predict(fs_pred_request_with_overridden_features)

如果所有获取的特征都被覆盖，则预测请求的行为与不集成特征存储时相同。比较 `fs_pred_request` 和 `default_pred_request` 的响应。

In [ ]:
endpoint.predict(default_pred_request)

调试

我们已经提供了一个身份模型服务器，使用[CPR](https://cloud.google.com/vertex-ai/docs/predictions/custom-prediction-routines)构建，以帮助调试您的特征获取配置。 CPR构建的身份模型服务器只是将请求体作为响应返回。这将允许您探索特征存储/预测集成将预测请求格式化为模型服务器。

该服务器已在此处发布 - `us-docker.pkg.dev/vertex-ai/sample-model-servers/cpr-identity-server:latest`，并使用以下`预测器`。

```python
from google.cloud.aiplatform.prediction.predictor import Predictor

class CprPredictor(Predictor):
    
    def __init__(self):
        return
    
    def load(self, artifacts_uri: str):
        """加载预处理器和模型工件。"""
        pass

    def predict(self, instances):
        """执行预测。"""
        return {"predictions": instances}
```

使用与上一个示例相同的功能获取配置文件，将其部署到 Identity model 服务器以进行调试。

首先将图像复制到 Artifacts Registry。

In [ ]:
!gcloud services enable artifactregistry.googleapis.com

In [ ]:
REPOSITORY = "feature-store-prediction-sample"
IMAGE = "cpr-identity-server"
CONTAINER_URI = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}"

In [ ]:
!gcloud artifacts repositories create {REPOSITORY} \
    --repository-format=docker \
    --location=$REGION

In [ ]:
!gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

In [ ]:
!docker pull us-docker.pkg.dev/vertex-ai/sample-model-servers/cpr-identity-server:latest
!docker tag us-docker.pkg.dev/vertex-ai/sample-model-servers/cpr-identity-server:latest $CONTAINER_URI
!docker push $CONTAINER_URI

现在上传并部署身份模型到Vertex Prediction进行调试。

In [ ]:
IDENTITY_MODEL_DISPLAY_NAME = "cpr_identity_with_feature_store_integration" + TIMESTAMP

identity_model = aiplatform.Model.upload(
    display_name=IDENTITY_MODEL_DISPLAY_NAME,
    artifact_uri=ARTIFACT_URI,
    serving_container_image_uri=CONTAINER_URI,
    sync=False,
)

identity_model.wait()

In [ ]:
identity_endpoint = identity_model.deploy(
    deployed_model_display_name=IDENTITY_MODEL_DISPLAY_NAME,
    traffic_split=TRAFFIC_SPLIT,
    machine_type=MACHINE_TYPE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    service_account=SERVICE_ACCOUNT,
)

让我们现在看看在先前示例中每个预测请求中实际发送到模型服务器的请求是什么样子。

In [ ]:
print("Predict request:")
print(fs_pred_request)
print("Request sent to model server:")
identity_endpoint.predict(fs_pred_request)

In [ ]:
print("Predict request:")
print(fs_pred_request_with_overridden_features)
print("Request sent to model server:")
identity_endpoint.predict(fs_pred_request_with_overridden_features)

In [ ]:
print("Predict request:")
print(default_pred_request)
print("Request sent to model server:")
identity_endpoint.predict(default_pred_request)

整理责任

In [ ]:
# delete feature store
mobile_gaming_feature_store.delete(sync=True, force=True)

In [ ]:
# delete Vertex AI resources
endpoint.undeploy_all()
endpoint.delete()
model.delete

identity_endpoint.undeploy_all()
identity_endpoint.delete()
identity_model.delete

In [ ]:
# Delete bucket
!gsutil -m rm -r $BUCKET_URI